In [7]:
# 定义关键字和符号的字典
keywords = {
    'int': 'KEY',
    'float': 'KEY',
    'const': 'KEY',
    'void': 'KEY',
    'if': 'KEY',
    'else': 'KEY',
    'while': 'KEY',
    'break': 'KEY',
    'continue': 'KEY',
    'return': 'KEY'
}

symbols = {
    '+': 'ADDOP',
    '-': 'SUBOP',
    '*': 'MULOP',
    '(': 'LPAREN',
    ')': 'RPAREN',
    '%': 'MODOP',
    '[': 'LBRACKET',
    ']': 'RBRACKET',
    '{': 'LBRACE',
    '}': 'RBRACE',
    ';': 'SEMICOLON',
    '<': 'RELOP',
    '>': 'RELOP',
    '=': 'RELOP',
    '!': 'LOGOP',
    '/': 'DIVOP'
}

# 定义状态转换函数
def transition_state(state, char):
    transitions = {
        0: {
            'alpha': 1,
            'digit': 2,
            'space': 0,
            'symbol': 0,
            'newline': 0,
            '&': 15,
            '|': 16,
            'other': 17
        },
        1: {
            'alpha': 1,
            'digit': 1,
            'underscore': 1,
            'other': 0
        },
        2: {
            'digit': 2,
            '.': 3,
            'e': 5,
            'other': 0
        },
        # 其他状态的转换函数...
        15: {
            '&': ('LOGOP', 'and'),
            'other': ('ERROR', 'logop_and')
        },
        16: {
            '|': ('LOGOP', 'or'),
            'other': ('ERROR', 'logop_or')
        },
        17: {
            'other': ('ERROR', 'unknown')
        }
    }

    char_type = 'other'
    if char.isalpha():
        char_type = 'alpha'
    elif char.isdigit():
        char_type = 'digit'
    elif char.isspace():
        char_type = 'space'
    elif char in symbols:
        char_type = 'symbol'
    elif char == '\n':
        char_type = 'newline'
    elif char == '_':
        char_type = 'underscore'

    return transitions[state].get(char_type, ('ERROR', 'unknown'))

# 打开输入文件和输出文件
with open("demofile.txt", "r") as f, open("result.txt", "a") as r:
    source_code = f.read()
    lines = source_code.split('\n')

    # 初始化状态和计数器
    state = 0
    line = 1
    word = 0
    chars = 0
    token = ''
    res = ''

    for line_text in lines:
        chars += len(line_text)

        for char in line_text:

            if isinstance(transition_state(state, char), tuple):
                token_type, token_value = transition_state(state, char)
                if token_type == 'other':
                    continue
                elif token_type == 'ERROR':
                    res += f"at line: {line} ('{token_type}', '{token_value}');"
                state, token = 0, ''
            else:
                if state == 0:
                    word += 1
                    token = ''
                token += char
                if state == 0:
                    is_keyword = keywords.get(token, None)
                    if is_keyword:
                        res += f"('{is_keyword}', {token});"
                    else:
                        res += f"('ID', {token});"
            # state = token_type

        if state != 0:
            if state == 1:
                is_keyword = keywords.get(token, None)
                if is_keyword:
                    res += f"('{is_keyword}', {token});"
                    state, token = 0, ''
                else:
                    res += f"('ID', {token});"
                    state, token = 0, ''
            elif state == 2:
                res += f"('NUM', {token});"
                state, token = 0, ''

        res += '\n'

    res += f"Total lines: {line}\n"
    res += f"Total words: {word}\n"
    res += f"Total chars: {chars}\n"

    r.write(res)

In [8]:
r = open("result.txt", "w")
r.write("")

0